# Modelo Relacional

Categoria(_id_, nome)

Regiao(_id_, nome)

Grupo(_id_, nome)

Receita(_id_, titulo, fonte, instrucoes, regiao_id, categoria_id)
  regiao_id chave estrangeira -> Regiao(id)
  categoria_id chave estrangeira -> Categoria(id)

Ingrediente(_id_, nome, grupo_id)
  grupo_id chave estrangeira -> Grupo(id)

ReceitaIngrediente(_receita_id_, _ingrediente_id_, qtde)
  _receita_id_ chave estrangeira -> Receita(id)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)

Nutriente(_id_, nome)

IngredienteNutriente(_ingrediente_id_, _nutriente_id_, valor, min, max)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)
  _nutriente_id_ chave estrangeira -> Nutriente(id)

ComponenteQuimico(_id_, nome, formula)

IngredienteComponenteQuimico(_ingrediente_id_, _componenteQuimico_id_)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)
  _componenteQuimico_id_ chave estrangeira -> ComponenteQuimico(id)


In [4]:
%defaultDatasource jdbc:h2:mem:db

In [5]:
CREATE TABLE Categoria (
  _id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
);


CREATE TABLE Regiao (
  _id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
);


CREATE TABLE Grupo (
  _id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
);


CREATE TABLE Receita (
  _id_ INT PRIMARY KEY,
  titulo VARCHAR(255) NOT NULL,
  fonte VARCHAR(255),
  instrucoes TEXT,
  regiao_id INT,
  categoria_id INT,
  FOREIGN KEY (regiao_id) REFERENCES Regiao(_id_),
  FOREIGN KEY (categoria_id) REFERENCES Categoria(_id_)
);


CREATE TABLE Ingrediente (
  _id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL,
  grupo_id INT,
  FOREIGN KEY (grupo_id) REFERENCES Grupo(_id_)
);


CREATE TABLE ReceitaIngrediente (
  _receita_id_ INT,
  _ingrediente_id_ INT,
  qtde INT,
  PRIMARY KEY (_receita_id_, _ingrediente_id_),
  FOREIGN KEY (_receita_id_) REFERENCES Receita(_id_),
  FOREIGN KEY (_ingrediente_id_) REFERENCES Ingrediente(_id_)
);

CREATE TABLE Nutriente (
  _id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
);


CREATE TABLE IngredienteNutriente (
  _ingrediente_id_ INT,
  _nutriente_id_ INT,
  valor DOUBLE,
  min DOUBLE,
  max DOUBLE,
  PRIMARY KEY (_ingrediente_id_, _nutriente_id_),
  FOREIGN KEY (_ingrediente_id_) REFERENCES Ingrediente(_id_),
  FOREIGN KEY (_nutriente_id_) REFERENCES Nutriente(_id_)
);


CREATE TABLE ComponenteQuimico (
  _id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL,
  formula VARCHAR(255)
);


CREATE TABLE IngredienteComponenteQuimico (
  _ingrediente_id_ INT,
  _componenteQuimico_id_ INT,
  PRIMARY KEY (_ingrediente_id_, _componenteQuimico_id_),
  FOREIGN KEY (_ingrediente_id_) REFERENCES Ingrediente(_id_),
  FOREIGN KEY (_componenteQuimico_id_) REFERENCES ComponenteQuimico(_id_)
);